Questions 14/15/16/17

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import random

class Net(nn.Module):
    def __init__(self, input_chan, kernel, strd, padding, output, pool):
        super().__init__()
        self.pool = pool
        self.conv1 = nn.Sequential(nn.Conv2d(input_chan, 16, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 81, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.out = nn.Linear(81, output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.pool == 'avg':
            x = F.avg_pool2d(x, kernel_size=x.size()[2:])
            # global_avg_pooling = torch.nn.AvgPool2d((x.shape[2],x.shape[3]))
            # x = global_avg_pooling(x)
            # x = torch.mean(x.view(x.size(0), x.size(1), -1), dim=2)
            # x = nn.AdaptiveAvgPool2d(x, (1,1))
            # x = F.avg_pool2d(x, (1, 1))
        if self.pool == 'max':
            x = F.max_pool2d(x, kernel_size=x.size()[2:])
            # global_max_pooling = torch.nn.MaxPool2d((x.shape[2],x.shape[3]))
            # x = global_max_pooling(x)
        x = torch.flatten(x, 1)
        output = self.out(x)
        return output

# class Net(nn.Module):
#     def __init__(self, input_chan, kernel, strd, padding, output, pool):
#         super().__init__()
#         self.pool_c = pool
#         self.pool = nn.MaxPool2d(2,2)
#         self.conv1 = nn.Conv2d(input_chan, 16, kernel, strd, padding)
#         self.conv2 = nn.Conv2d(16, 32, kernel, strd, padding)
#         self.conv3 = nn.Conv2d(32, 81, kernel, strd, padding)
#         self.out = nn.Linear(81, output)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         if self.pool_C == 'max':
#             x = F.max_pool2d(x, kernel_size=x.size()[2:])
#         if self.pool_C == 'avg':
#             x = F.avg_pool2d(x, kernel_size=x.size()[2:])
#         x = torch.flatten(x, 1)
#         output = self.out(x)
#         return output

In [ ]:
class NetFixed(nn.Module):
    def __init__(self, input_chan, kernel, stride, padding, output):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(input_chan, 16, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.out = nn.Linear(64*3*3, output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        output = self.out(x)
        return output

Question15

In [ ]:
def get_parameters(model):
    count=0
    for param in list(model.parameters()):
        n=1
        for s in list(param.size()):
            n = n*s
        count += n
    return count

In [ ]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
model_fixed = NetFixed(input_chan, kernel, stride, padding, output)
print('fixed model parameters = ', get_parameters(model_fixed))
model_mixed = Net(input_chan, kernel, stride, padding, output, pool='avg')
print('mixed model parameters = ', get_parameters(model_mixed))

Question14

In [ ]:
def validate(net, loader):
    net.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loader['val_set']:
            output = net(x)
            _,pred_y = torch.max(output, dim = 1)
            correct += (pred_y == y).float().sum()

    print('accuracy on validation set', (correct / 10000)*100, '%')
    return (correct / 10000)*100

def loader_loop(net, loader_list, epochs, loss_f, opt):
    train_loss = []
    epoch_list = []
    acc_list = []

    for i in range(epochs):
        print('epoch = ', i)
        for loader in loader_list:
            for j, (x, y) in enumerate(loader['train_set']):
                opt.zero_grad()
                x_batch = x
                y_batch = y
                output = net.forward(x_batch)
                loss = loss_f(output, y_batch)
                train_loss.append(loss)
                if j % 1000 == 0:
                    print('loss:', loss.item())
                loss.backward()
                opt.step() 
            validate(net, loader)
        epoch_list.append(i)
        acc_list.append(validate(net, random.choice(loader_list)))
    return [train_loss, epoch_list, acc_list]

def image_folder(path):
    training_data = torchvision.datasets.ImageFolder(path+'mnist-varres/train', transform =  transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]))
    test_set= torchvision.datasets.ImageFolder(path+'mnist-varres/test', transform =  transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]))
    trainsize = round(0.8*len(training_data))
    train_set, val_set = torch.utils.data.random_split(training_data, [trainsize, len(training_data)-trainsize]) 
    print(len(train_set), len(val_set))
    return [train_set, val_set, test_set]

def get_data(path1, path2, path3):
    data1 = image_folder(path1)
    data2 = image_folder(path2)
    data3 = image_folder(path3)
    return data1, data2, data3

def get_loaders(batch_size, train_set, val_set, test_set):
    loaders = {'train_set' : DataLoader(train_set, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=1),
               'val_set' : DataLoader(val_set, 
                                          batch_size=len(val_set), 
                                          shuffle=True, 
                                          num_workers=1),
                'test_set'  : DataLoader(test_set, 
                                          batch_size=len(test_set), 
                                          shuffle=False, 
                                          num_workers=1)}
    return loaders

In [ ]:
path1 = '32x32/'
path2 = '48x48/'
path3 = '64x64/'
data32, data48, data64 = get_data(path1, path2, path3)


batch_size = 16
loaders32 = get_loaders(batch_size, data32[0], data32[1], data32[2])
loaders48 = get_loaders(batch_size, data48[0], data48[1], data48[2])
loaders64 = get_loaders(batch_size, data64[0], data64[1], data64[2])
loader_list = [loaders32, loaders48, loaders64]

Question 16

In [ ]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
epochs = 20
lr = 0.0001
loss_f = nn.CrossEntropyLoss()
net_avg = Net(input_chan, kernel, stride, padding, output, pool='avg')
opt = optim.Adam(net_avg.parameters(), lr)
results_avg = loader_loop(net_avg, loader_list, epochs, loss_f, opt)
# net_max = Net(input_chan, kernel, stride, padding, output, pool='max')
# loss_f = nn.CrossEntropyLoss()
# opt = optim.Adam(net_max.parameters(), lr)
# results_max = loader_loop(net_max, loader_list, epochs, loss_f, opt)

In [ ]:
y1 = results_avg[2]
x1 = results_avg[1]
plt.plot(x1, y1, label = 'AvgPool')
y2 = results_max[2]
x2 = results_max[1]
plt.plot(x2, y2, label = 'MaxPool')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc='best')
plt.savefig('figures/q16_N=81')
plt.show

Question 17

In [ ]:
#tuning on learning rate, batchsize and...?

In [ ]:
def validate(net, loaders):
    net.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loaders['val_set']:
            output = net(x)
            _,pred_y = torch.max(output, dim = 1)
            correct += (pred_y == y).float().sum()

    print('accuracy on validation set', (correct / 10000)*100, '%')
    return (correct / 10000)*100

def train(net, loaders, epochs, loss_f, opt):
    train_loss = []
    epoch_list = []
    acc_list = []

    for i in range(epochs):
        print('epoch = ', i)
        for j, (x, y) in enumerate(loaders['train_set']):
            opt.zero_grad()
            x_batch = x
            y_batch = y
            output = net.forward(x_batch)
            loss = loss_f(output, y_batch)
            train_loss.append(loss)
            if j % 1000 == 0:
                print('loss:', loss.item())
            loss.backward()
            opt.step() 
        epoch_list.append(i)
        acc_list.append(validate(net, loaders))
    return train_loss, epoch_list, acc_list

In [ ]:
def get_loaders(batch_size, train_set, val_set, test_set):
    loaders = {'train_set' : DataLoader(train_set, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=1),
               'val_set' : DataLoader(val_set, 
                                          batch_size=len(val_set), 
                                          shuffle=True, 
                                          num_workers=1),
                'test_set'  : DataLoader(test_set, 
                                          batch_size=len(test_set), 
                                          shuffle=False, 
                                          num_workers=1)}
    return loaders

In [ ]:
one_size = transforms.Compose(
                    [
                    transforms.Resize((28,28)),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1308,), (0.3016,)) 
                    ])

training_data = torchvision.datasets.ImageFolder('mnist-varres/train', transform=one_size)
test_set= torchvision.datasets.ImageFolder('mnist-varres/test', transform=one_size)
train_set, val_set = torch.utils.data.random_split(training_data, [50000, 10000]) 

In [ ]:
input_chan = 1 
output = 10
kernel = 3
stride = 1
padding = 1
lr = 0.0001
net = Net(input_chan, kernel, stride, padding, output, pool='max')
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr)

epochs = 5
batch_size = 16
loaders = get_loaders(batch_size, train_set, val_set, test_set) 
train_loss, epoch_list, acc_list = train(net, loaders, epochs, loss_f, opt)